In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchmetrics import MeanAbsolutePercentageError as MAPE
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import datetime
import sys
import Dataset_Class as DC


# USE_CUDA = torch.cuda.is_available()
# DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
DEVICE = 'cpu'
RANDOM_SEED = 2023
EPOCHS = 50000
LEARNING_RATE = 0.000001
BATCH_SIZE = 16


DEVICE = 'cpu'



class Net(nn.Module):
    def __init__(self, num_of_features, **model_config):
        super(Net, self).__init__()
        self.model_type = model_config['case']
        if model_config['case'] == 1:
            self.hidd_dim = 256   # good
            self.hidden_dim = 36
        elif model_config['case'] == 2:
            self.hidd_dim = 240
            self.hidden_dim = 36
        elif model_config['case'] == 3:
            self.hidd_dim = 240
            self.hidden_dim = 72
        elif model_config['case'] == 4:
            self.hidd_dim = 120
            self.hidden_dim = 36
            
        self.fc1 = nn.Linear(num_of_features, self.hidd_dim)
        self.fc2 = nn.Linear(self.hidd_dim, self.hidden_dim)
        self.fc3 = nn.Linear(self.hidden_dim, 24)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        output = self.fc3(x)
        return output
     

# Set seed for reproducibility
def set_seed(seed):
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(RANDOM_SEED)
    random.seed(RANDOM_SEED)


# load the data and split into X and Y
def load_data():
    X_load = pd.read_csv('./processed_data/load/X_load.csv', index_col=0)
    Y_load = pd.read_csv('./processed_data/load/Y_load.csv', index_col=0)
    X = torch.FloatTensor(X_load.values)
    Y = torch.FloatTensor(Y_load.values)
    return X, Y


# split data into mini_train, valid, train, test
def split_data(X, Y, batch_size, data_len, train_pie, mini_train_pie):
    train_size = int(data_len * train_pie)
    mini_train_size = int(train_size * mini_train_pie)
    
    train_data = DC.CustomDataset(X[:train_size], Y[:train_size])
    test_data = DC.CustomDataset(X[train_size:], Y[train_size:])
    mini_train_data = DC.CustomDataset(X[:mini_train_size], Y[:mini_train_size])
    valid_data = DC.CustomDataset(X[mini_train_size:train_size], Y[mini_train_size:train_size])
    mini_train_dataloader = DataLoader(mini_train_data, batch_size = batch_size, shuffle = True)
    valid_dataloader = DataLoader(valid_data, batch_size = len(valid_data), shuffle = False)
    train_dataloader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
    test_dataloader = DataLoader(test_data, batch_size = len(test_data), shuffle = False)
    
    return mini_train_dataloader, valid_dataloader, train_dataloader, test_dataloader, mini_train_size, train_size



def plot(i, length, output, Y, fig_size, title, font_size, save_path):
    fig = plt.figure(figsize=fig_size)
    plt.title(title, fontsize = font_size)
    plt.xlabel('Time (h)', fontsize = font_size)
    plt.ylabel('Load', fontsize = font_size)
    plt.plot(Y.detach().numpy()[i:i+length,:].reshape(-1), c='blue', label = 'Actual data')
    plt.plot(output.detach().numpy()[i:i+length,:].reshape(-1), c='red', label = 'forecast data')
    plt.legend(loc='lower right', fontsize = 13)
    plt.savefig(save_path)

     
def train(model, train_dataloader, optimizer, criterion):
    model.train()
    loss_sum = 0.0
    for (x, y) in train_dataloader:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        
        output = model(x)
        train_loss = criterion(output, y)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        loss_sum += train_loss
    
    return (loss_sum/len(train_dataloader.dataset)/24).item() # loss of each epoch


def evaluate(model, valid_dataloader):
    model.eval()
    with torch.no_grad():
        for (x, y) in valid_dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
                
            output = model(x)
            
        return output, y


set_seed(RANDOM_SEED)
model_case = 1
num_of_features = 24
mae = nn.L1Loss()
mape = MAPE()


# data loading
# X: 210104-211229, Y: 210105-211230 / 173*50
X, Y = load_data()
dataset = DC.CustomDataset(X, Y)
data_len = len(dataset)
mini_train_dataloader, valid_dataloader, train_dataloader, test_dataloader, mini_train_size, train_size =  split_data(X, Y, BATCH_SIZE, data_len, 0.8, 0.8)
# 110개   210104 ~ 210816
# 28개    210817 ~ 211013
# 138개   210104 ~ 211013
# 35개    211014 ~ 211229



model_case = 1
model_config = {'case': model_case}


# load state_dict from model.pt file
model = Net(24, **model_config)
model.load_state_dict(torch.load('./experiment_outputs/load_forecast/models/1_50000_1e-06_16_0322_1332.pt'))


<All keys matched successfully>

In [3]:
test_output, test_y = evaluate(model, test_dataloader)

In [12]:
y = test_y[:, 23:]
print(y)

tensor([[776.7000],
        [712.9000],
        [700.1000],
        [665.0000],
        [602.3000],
        [665.8000],
        [679.4000],
        [657.0000],
        [689.3000],
        [711.4000],
        [702.6000],
        [664.3000],
        [691.8000],
        [685.0000],
        [714.0000],
        [715.5000],
        [691.1000],
        [693.7000],
        [770.3000],
        [746.7000],
        [721.6000],
        [687.5000],
        [805.0000],
        [824.9000],
        [801.1000],
        [744.9000],
        [708.0000],
        [863.6000],
        [756.9000],
        [742.7000],
        [864.1000],
        [808.8000],
        [896.1000],
        [869.2000],
        [856.0000]])


In [18]:
for i in range(len(y)):
    print(i, y[i].item())

0 776.7000122070312
1 712.9000244140625
2 700.0999755859375
3 665.0
4 602.2999877929688
5 665.7999877929688
6 679.4000244140625
7 657.0
8 689.2999877929688
9 711.4000244140625
10 702.5999755859375
11 664.2999877929688
12 691.7999877929688
13 685.0
14 714.0
15 715.5
16 691.0999755859375
17 693.7000122070312
18 770.2999877929688
19 746.7000122070312
20 721.5999755859375
21 687.5
22 805.0
23 824.9000244140625
24 801.0999755859375
25 744.9000244140625
26 708.0
27 863.5999755859375
28 756.9000244140625
29 742.7000122070312
30 864.0999755859375
31 808.7999877929688
32 896.0999755859375
33 869.2000122070312
34 856.0


In [ ]:
# interval_1 = [0]
# interval_2 = []
# interval_3 = []